# Drone Detection - Architecture Comparison Training

Trains YOLO11-L (CNN) and RT-DETR-L (Attention) on subsets of 50-3000 images, saving checkpoints and metrics for later analysis.

---
## Configuration

In [ ]:
import os
import sys
import shutil
import random
import tempfile
from pathlib import Path

import yaml
import torch
import pandas as pd
import ultralytics
from ultralytics import YOLO, RTDETR

# Paths (relative to notebooks/ directory)
PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data' / 'dataset'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'

MODELS = {
    'yolo11l': 'yolo11l.pt',
    'rtdetr-l': 'rtdetr-l.pt',
}

DATASET_SIZES = [50, 100, 200, 500, 1500, 3000]

EPOCHS = 40
BATCH_SIZE = 16
IMAGE_SIZE = 640
LEARNING_RATE = 0.001
OPTIMIZER = 'AdamW'
CHECKPOINT_INTERVAL = 10
NUM_WORKERS = 4
RANDOM_SEED = 42

USE_WANDB = False
WANDB_PROJECT = 'drone-detection-comparison'

---
## Environment

In [ ]:
print(f"Python:      {sys.version.split()[0]}")
print(f"PyTorch:     {torch.__version__}")
print(f"Ultralytics: {ultralytics.__version__}")
print(f"CUDA:        {torch.cuda.is_available()}")
print(f"MPS:         {torch.backends.mps.is_available()}")

if torch.cuda.is_available():
    print(f"GPU:         {torch.cuda.get_device_name(0)}")
    DEVICE = 0
elif torch.backends.mps.is_available():
    print(f"GPU:         Apple Silicon (MPS)")
    DEVICE = 'mps'
else:
    print("GPU:         None (CPU only)")
    DEVICE = 'cpu'

In [ ]:
train_imgs = list((DATA_DIR / 'train' / 'images').glob('*.jpg'))
valid_imgs = list((DATA_DIR / 'valid' / 'images').glob('*.jpg'))
test_imgs = list((DATA_DIR / 'test' / 'images').glob('*.jpg'))
print(f"Train: {len(train_imgs)} | Valid: {len(valid_imgs)} | Test: {len(test_imgs)}")

---
## Subset Creation

In [ ]:
def create_subset(src_dir, dst_dir, n_samples, seed=42):
    """Copy n_samples training images + all valid/test to dst_dir"""
    random.seed(seed)

    train_imgs = list((src_dir / 'train' / 'images').glob('*.jpg'))
    selected = random.sample(train_imgs, min(n_samples, len(train_imgs)))

    for img in selected:
        (dst_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
        (dst_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
        shutil.copy(img, dst_dir / 'train' / 'images' / img.name)
        lbl = src_dir / 'train' / 'labels' / (img.stem + '.txt')
        if lbl.exists():
            shutil.copy(lbl, dst_dir / 'train' / 'labels' / lbl.name)

    # copy full valid/test sets
    for split in ['valid', 'test']:
        for img in (src_dir / split / 'images').glob('*.jpg'):
            (dst_dir / split / 'images').mkdir(parents=True, exist_ok=True)
            (dst_dir / split / 'labels').mkdir(parents=True, exist_ok=True)
            shutil.copy(img, dst_dir / split / 'images' / img.name)
            lbl = src_dir / split / 'labels' / (img.stem + '.txt')
            if lbl.exists():
                shutil.copy(lbl, dst_dir / split / 'labels' / lbl.name)

    with open(dst_dir / 'data.yaml', 'w') as f:
        yaml.dump({
            'train': str(dst_dir / 'train' / 'images'),
            'val': str(dst_dir / 'valid' / 'images'),
            'test': str(dst_dir / 'test' / 'images'),
            'nc': 1, 'names': ['drone']
        }, f)

    return len(selected)

---
## W&B Setup (Optional)

In [ ]:
if USE_WANDB:
    import wandb
    wandb.login()
    ultralytics.settings.update({'wandb': True})

---
## Training Function

In [ ]:
if USE_WANDB:
    import wandb


def train_model(model_name, model_weights, data_yaml, output_dir, n_images):
    print(f"\n{model_name} | {n_images} images | -> {output_dir}")
    
    output_dir.mkdir(parents=True, exist_ok=True)
    
    if 'rtdetr' in model_weights.lower():
        model = RTDETR(model_weights)
    else:
        model = YOLO(model_weights)
    
    if USE_WANDB:
        wandb.init(
            project=WANDB_PROJECT,
            name=f"{model_name}_{n_images}",
            config={
                'model': model_name,
                'weights': model_weights,
                'dataset_size': n_images,
                'epochs': EPOCHS,
                'batch_size': BATCH_SIZE,
                'image_size': IMAGE_SIZE,
                'learning_rate': LEARNING_RATE,
                'optimizer': OPTIMIZER,
            },
            reinit=True,
        )
    
    results = model.train(
        data=str(data_yaml),
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMAGE_SIZE,
        lr0=LEARNING_RATE,
        optimizer=OPTIMIZER,
        patience=0,  # no early stopping - we want all epochs for curves
        save_period=CHECKPOINT_INTERVAL,
        workers=NUM_WORKERS,
        device=DEVICE,
        project=str(output_dir),
        name='train',
        exist_ok=True,
        verbose=True,
    )
    
    if USE_WANDB:
        wandb.finish()
    
    return output_dir / 'train'

---
## Run Training

In [ ]:
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

total_runs = len(DATASET_SIZES) * len(MODELS)
current_run = 0

print(f"Starting {total_runs} runs: {list(MODELS.keys())} x {DATASET_SIZES}")

for n_images in DATASET_SIZES:
    print(f"\n--- Dataset size: {n_images} ---")
    
    # create temp subset - both models train on same subset
    subset_dir = Path(tempfile.mkdtemp()) / f'subset_{n_images}'
    actual_size = create_subset(DATA_DIR, subset_dir, n_images, seed=RANDOM_SEED)
    print(f"Created subset with {actual_size} training images")
    
    data_yaml = subset_dir / 'data.yaml'
    
    for model_name, model_weights in MODELS.items():
        current_run += 1
        print(f"\nRun {current_run}/{total_runs}")
        
        output_dir = ARTIFACTS_DIR / f'{model_name}_{n_images}'
        
        train_model(
            model_name=model_name,
            model_weights=model_weights,
            data_yaml=data_yaml,
            output_dir=output_dir,
            n_images=actual_size,
        )
    
    shutil.rmtree(subset_dir.parent, ignore_errors=True)  # cleanup temp

print(f"\nAll done.")

---
## Summary

In [ ]:
for run_dir in sorted(ARTIFACTS_DIR.iterdir()):
    if run_dir.is_dir() and run_dir.name != '__pycache__':
        print(f"\n{run_dir.name}/")
        
        train_dir = run_dir / 'train'
        if train_dir.exists():
            weights_dir = train_dir / 'weights'
            if weights_dir.exists():
                for pt_file in sorted(weights_dir.glob('*.pt')):
                    size_mb = pt_file.stat().st_size / 1e6
                    print(f"  weights/{pt_file.name} ({size_mb:.1f} MB)")
            
            results_csv = train_dir / 'results.csv'
            if results_csv.exists():
                df = pd.read_csv(results_csv)
                print(f"  results.csv ({len(df)} epochs)")

---
## Next Steps

Run `02_analyze_results.ipynb` to load metrics, evaluate on the test set, and generate comparison plots.